# Calculation 04 - Stacking fault energies
The following notebook will calculate the intrinsic and extrinsic stacking fault energies of Nickel-based EAM and MEAM potentials. The first cell needs updating depending on the potential file you are using and the compound you want to run the calculation for.\
At the end of the notebook, there is a "fast run" which will allow you to run this notebook for all potentials and both stacking faults in one go.

## 📌 Specify filenames and element related data
For any potential, please specify the filename of the potential file as follows:\
`potentials/filename.eam`\
`potentials/filename.eam.fs`\
`potentials/filename.eam.alloy`\
`potentials/filename.meam`

Then uncomment the respective lines, depending on which compound you want to run the calculation for.

**Note:** to call Non-LAMMPS variables within a LAMMPS command, please use an f-string. To call LAMMPS variables, instead of using an f-string, just use curly braces with a leading dollar sign - like this: `${x}`.

In [ ]:
potentialpath = "potentials/Ni1986.eam"
latticeconstant = 3.52
stackingfault = "extrinsic"
outputfile = "data/04_stacking-faults_"+stackingfault+".dat"

## ⬇️ Import PyLammps module

In [ ]:
from lammps import PyLammps
lmp = PyLammps()

lmp.clear()

## ⬇️ Add parameters for MEAM potentials
There is no need to change the following cell. It defines two parameters that are necessary for reading in MEAM potentials.\
**Please do note that the libraryelements variable has to contain the element symbols in the same order as they appear in the library file!**

In [ ]:
if "meam" in potentialpath:
    libraryfile = str(potentialpath.replace('.meam', '.library.meam'))
    
    if "NiAlCo2" in potentialpath:
        libraryelements = "Ni Al Co"
    elif "NiAl2" in potentialpath:
        libraryelements = "Al Ni"
    else:
        libraryelements = "Ni"

## ⬇️ Initialize simulation
`units` sets the unit system for this simulation. "metal" units are Angstroms for distance, eV for energy, etc.\
`dimension` is self-explanatory and can either be 2 or 3.\
`boundary` sets the boundary conditions in x-, y- and z-direction. Here, they are periodic in the x and y dimensions, but non-periodic and shrink-wrapped in the z dimension.\
`atom_style` specifies additional attributes depending on the style - "atomic" doesn't have any.

In [ ]:
lmp.units('metal')
lmp.dimension('3')
lmp.boundary('p p s')
lmp.atom_style('atomic')

## ⬇️ Create atoms
`lattice` specifies the predefined or customized lattice and the respective lattice constant.\
`region` specifies the simulation cell as a geometric shape, like a sphere, cylinder or block. The default coordinate unit is a "lattice parameter", meaning each coordinate is an integer factor times the lattice parameter specified in the `lattice` command. "cell" is the user-assigned ID for this specific region, but could also be called "1", "a" or anything else. For periodic boundary conditions, the cell dimensions should be multiples of the lattice constant to prevent unwanted overlaps (as explained in the documentation for the `create_atoms` command).\
`create_box` actually creates the simulation cell as defined in the `region` command. The integer defines the number of element types that will be used, the string defines the region in which the cell will be created.\
`create_atoms` adds atoms to the lattice within a specific region, so this has to happen after defining and creating the simulation box. The integer defines the number of element types used.\
`replicate` would allow to change the size of the simulation, and in this position is equivalent to changing the coordinates in the `region` command.
`mass` does not need to specified for any EAM or MEAM potential since it is used from the potential file itself.

In [ ]:
lmp.variable(f'a_to_b equal -1*({latticeconstant}/sqrt(6))')
lmp.variable(f'a_to_c equal 1*({latticeconstant}/sqrt(6))')

lmp.variable(f'x equal {latticeconstant}*20*sqrt(6)/4')
lmp.variable(f'y equal {latticeconstant}*10*sqrt(2)/2')

lmp.variable('zero equal -0.001')

if stackingfault == "intrinsic":
    lmp.variable(f'z1 equal {latticeconstant}*8.5*sqrt(3)/3')
    lmp.variable(f'z2 equal {latticeconstant}*17.5*sqrt(3)/3') 

    lmp.lattice(f'fcc {latticeconstant} orient x 1 1 2 orient y -1 1 0 orient z -1 -1 1')

    lmp.region('bottom block ${zero} ${x} ${zero} ${y} ${zero} ${z1} units box')
    lmp.region('top block ${zero} ${x} ${zero} ${y} ${z1} ${z2} units box')

    lmp.region('cell block 0 ${x} 0 ${y} 0 ${z2} units box')

    lmp.create_box('1 cell')

    lmp.create_atoms('1 region bottom')
    lmp.create_atoms('1 region top')

elif stackingfault == "extrinsic":
    lmp.variable(f'z1 equal {latticeconstant}*8.5*sqrt(3)/3')
    lmp.variable(f'z2 equal {latticeconstant}*9.5*sqrt(3)/3')
    lmp.variable(f'z3 equal {latticeconstant}*17.5*sqrt(3)/3')

    lmp.lattice(f'fcc {latticeconstant} orient x 1 1 2 orient y -1 1 0 orient z -1 -1 1')

    lmp.region('bottom block ${zero} ${x} ${zero} ${y} ${zero} ${z1} units box')
    lmp.region('middle block ${zero} ${x} ${zero} ${y} ${z1} ${z2} units box')
    lmp.region('top block ${zero} ${x} ${zero} ${y} ${z2} ${z3} units box')

    lmp.region('cell block 0 ${x} 0 ${y} 0 ${z3} units box')

    lmp.create_box('1 cell')

    lmp.create_atoms('1 region bottom')
    lmp.create_atoms('1 region middle')
    lmp.create_atoms('1 region top')

## ⬇️ Read in potential file
`pair_style` specifies what kind of interatomic potential will be used.\
`pair_coeff` specifies the potential file to be read in. The input parameters change depending on the potential style. that the pair potential coefficients are stored in. The first two integers define the force field coefficients between atom pairs. The asterisks include all atom types.\
`neighbor` and `neigh_modify` would set parameters for the neighbor lists - the default parameters when the commands are not used can be found in the LAMMPS documentation.

In [ ]:
if "meam" in potentialpath:
    lmp.pair_style('meam')
    lmp.pair_coeff(f'* * {libraryfile} {libraryelements} {potentialpath} Ni')
elif "eam.alloy" in potentialpath:
    lmp.pair_style('eam/alloy')
    lmp.pair_coeff(f'* * {potentialpath} Ni')
elif "eam.fs" in potentialpath:
    lmp.pair_style('eam/fs')
    lmp.pair_coeff(f'* * {potentialpath} Ni')
else:
    lmp.pair_style('eam')
    lmp.pair_coeff(f'* * {potentialpath}')

## ⬇️ Define computes
`compute`s define "formulas" that can later on be computed at each timestep of e.g. a thermodynamic run, minimization or dump output.\
The first one allows to store the potential energy for each atom during any "run".\
The second one uses the first one, summing up the potential energies of all atoms in the system.\
The third one allows to calculate the centro-symmetry parameter.

In [ ]:
lmp.compute('Epot all pe/atom')
lmp.compute('Epotall all reduce sum c_Epot')
lmp.compute('Csym all centro/atom fcc')

## ⬇️ Define thermo settings
`min_style` would set which minimization algorithm should be used for the minimization. The default value is `cg` - conjugate gradient.\
`thermo` sets the timestep interval at which computes shall be performed and printed during the run.\
`thermo_style` allows to specify what information shall be printed at each of those timesteps - in this case, it is customized to calculate and print the timestep, the length of the simulation box in x, y and z direction, and the total potential energy of the system.

In [ ]:
lmp.thermo(10)
lmp.thermo_style('custom step lx ly lz pe c_Epotall')

## ⬇️ Define dump file
`dump` will print the defined properties in a dump file at the desired timestep interval. With these files, the simulation can be visualized using software like OVITO.

In [ ]:
lmp.dump('dump all cfg 1000 dump/04-'+stackingfault+'.*.cfg mass type xs ys zs c_Csym c_Epot')
lmp.dump_modify('dump element Ni')

## ⬇️ Run minimization I
`minimize`s the energy of the simulation cell, printing out the desired information (as specified in `thermo_style`) at every `thermo(10)`th timestep.

In [ ]:
lmp.minimize('1e-10 1e-10 10000 100000')

## ⬇️ Define variable formulas
This section defines some `variable`s by assigning variables to a formula that are not yet evaluated.\
The first one is the total energy of the simulation cell E, the second one the area of the stacking fault plane. 

In [ ]:
lmp.variable('E equal "c_Epotall"')
lmp.variable('A equal "lx*ly*1e-20"')

## ⬇️ Calculate properties I
Here, the formula defined above is calculated with the current value of the formula variable and stored in a new variable.

In [ ]:
lmp.variable('Ei equal $E')

## ⬇️ Generate stacking fault
`group` assigns all atoms in a region (specified in "Create atoms") to the respective group. Even if atoms move out of the original region, they will remain in the group they were assigned to.\
`displace_atoms` actually displaces the atom(s) in the defined group.

In [ ]:
if stackingfault == "intrinsic":
    lmp.group('top region top')

    lmp.displace_atoms('top move ${a_to_b} 0 0 units box')
    
elif stackingfault == "extrinsic":
    lmp.group('top region top')
    lmp.group('middle region middle')

    lmp.displace_atoms('middle move ${a_to_b} 0 0 units box')
    lmp.displace_atoms('top move ${a_to_c} 0 0 units box')

## ⬇️ Define fix
`fix` applies specific operations to a specified group of atoms (in this case, `all` of them). Here, the operation is to set the  force on all atoms in x- and y-direction to zero and leave it unchanged in z-direction, in which the stacking fault was introduced.

In [ ]:
lmp.fix('1 all setforce 0 0 NULL')

## ⬇️ Run minimization II
`minimize`s the energy of the simulation cell, using the parameters defined in `fix(1)` and printing out the desired information (as specified in `thermo_style`) at every `thermo(10)`th timestep.

In [ ]:
lmp.minimize('1e-10 1e-10 10000 100000')

## ⬇️ Calculate properties II
Again, the formula defined above is calculated with the current value of the formula variable and stored in a new variable.
A second variable, calculating the stacking fault energy, is defined and will be calculated in the `print` command with the current variable values.

In [ ]:
lmp.variable('Ef equal $E')

lmp.variable('J_to_eV equal 1.60217657e-16')
lmp.variable('Esf equal ((${Ef}-${Ei})*${J_to_eV})/${A}')

## ⬇️ Undump
Close dump file.

In [ ]:
lmp.undump('dump')

## ⬇️ Print desired output
This section `print`s these values to the output file, for which you chose a filename in the beginning.

In [ ]:
potentialname = potentialpath.replace('potentials/', '')

lmp.print(f'"{potentialname}'+', ${Ei}, ${Ef}, ${Esf}" append '+f'{outputfile} screen no')

🏁**ALL DONE!**🏁

🏁**FAST RUN DONE!**🏁